In [11]:
from joblib import load, dump
import pandas as pd
import os

# Load data
prices = load("C:/Users/DELL/Documents/Data Science and Analytics/Mini Capstone/minic/Database/prices_wif.joblib")
txns = load("C:/Users/DELL/Documents/Data Science and Analytics/Mini Capstone/minic/Database/whales_wif.joblib")

# Convert prices['timestamp'] if not already datetime
prices['timestamp'] = pd.to_datetime(prices['timestamp'], utc=True)

# Convert whale data day to datetime (handle the date format properly)
txns['day'] = pd.to_datetime(txns['day'], utc=True)

# Create a date column for merging (extract date from hourly timestamps)
prices['date'] = prices['timestamp'].dt.date
txns['date'] = txns['day'].dt.date

# Prepare whale data for merging
whale_data = txns.set_index('date')['whale_tx_count']

# Merge (this will add the same daily whale count to all hours of that day)
df = prices.set_index('date').join(whale_data, how='left')

# Reset index to get date back as column, but keep timestamp as well
df = df.reset_index()
df = df.set_index('timestamp')

# Fill missing whale tx counts with 0
df['whale_tx_count'] = df['whale_tx_count'].fillna(0)

# Drop the date column if not needed
df = df.drop('date', axis=1)

# Preview
print(df.head())

# Save to disk
save_dir = "C:/Users/DELL/Documents/Data Science and Analytics/Mini Capstone/minic/Database"
os.makedirs(save_dir, exist_ok=True)
dump(df, os.path.join(save_dir, "merged_wif_data.joblib"))

                                     price  whale_tx_count
timestamp                                                 
2025-07-07 15:03:37.852000+00:00  0.841319              27
2025-07-07 16:03:27.405000+00:00  0.847040              27
2025-07-07 17:03:30.210000+00:00  0.847720              27
2025-07-07 18:03:44.796000+00:00  0.845687              27
2025-07-07 19:03:45.507000+00:00  0.846087              27


['C:/Users/DELL/Documents/Data Science and Analytics/Mini Capstone/minic/Database\\merged_wif_data.joblib']